# Lab 10

#### Imports

In [1]:
import os
from functional import seq
from IPython.core.display import HTML
from fastai.text import *

## Setup

In [2]:
!tree blobs.bak/work/up_low50k/

blobs.bak/work/up_low50k/
├── models
│   └── fwd_v50k_finetune_lm_enc.h5
└── tmp
    ├── sp-50k.model
    └── sp-50k.vocab

2 directories, 3 files


In [3]:
fastai_model_path = "./blobs.bak/work/up_low50k/models/fwd_v50k_finetune_lm_enc.h5"
sentencepiece_model_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.model"
sentencepiece_vocab_path = "./blobs.bak/work/up_low50k/tmp/sp-50k.vocab"

In [4]:
def exists_or_ex(path):
    file = open(path,"r")
    file.close()
_ = [exists_or_ex(path) for path in [fastai_model_path, sentencepiece_model_path, sentencepiece_vocab_path]]

### Coloring

In [5]:
from termcolor import colored


def highlighted(
        highlight,
        whole_text,
):
    return whole_text.replace(
        highlight, colored(highlight, color="green"))

### Sentencepiece

In [28]:
import sentencepiece as spm
spm_processor = spm.SentencePieceProcessor()
spm_processor.Load(sentencepiece_model_path)


# spm_processor.LoadVocabulary(sentencepiece_vocab_path,threshold= 100)
spm_processor.SetEncodeExtraOptions("bos:eos")
spm_processor.SetDecodeExtraOptions("bos:eos")

True

### Loading model

In [29]:
def get_lm(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token, bidir=False,
           tie_weights=True, qrnn=False):
    rnn_enc = MultiBatchRNN(bptt, max_seq, n_tok, emb_sz, n_hid, n_layers, pad_token=pad_token, bidir=bidir, qrnn=qrnn)
    enc = rnn_enc.encoder if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(n_tok, emb_sz, 0, tie_encoder=enc))

In [30]:
torch.cuda.set_device(0)

UNK_ID = 0
PAD_ID = 1
BOS_ID = 2
EOS_ID = 3
UP_ID  = 4
bs=22

em_sz,nh,nl = 400 ,1150,4

bptt=5
vs = len(spm_processor)

lm = get_lm(bptt, 1000000, vs, em_sz, nh, nl, PAD_ID)
lm = to_gpu(lm)
load_model(lm[0],fastai_model_path)
lm.reset()
lm.eval()

SequentialRNN(
  (0): MultiBatchRNN(
    (encoder): Embedding(50000, 400, padding_idx=1)
    (encoder_with_dropout): EmbeddingDropout(
      (embed): Embedding(50000, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDrop(
        (module): LSTM(400, 1150)
      )
      (1): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (2): WeightDrop(
        (module): LSTM(1150, 1150)
      )
      (3): WeightDrop(
        (module): LSTM(1150, 400)
      )
    )
    (dropouti): LockedDropout()
    (dropouths): ModuleList(
      (0): LockedDropout()
      (1): LockedDropout()
      (2): LockedDropout()
      (3): LockedDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=50000, bias=False)
    (dropout): LockedDropout()
  )
)

In [31]:
class LMTextDataset(Dataset):
    def __init__(self, x):
        self.x = x

    def __getitem__(self, idx):
        sentence = self.x[idx]
        return sentence[:-1], sentence[1:]

    def __len__(self):
        return len(self.x)

## Using model

In [34]:
def next_tokens(ids_, model, toks_at_once):  #toks_at_once wont be used
    ids = [np.array(ids_)]
    test_ds = LMTextDataset(ids)
    test_samp = SortSampler(ids, key=lambda x: len(ids[x]))
    dl = DataLoader(test_ds,
                    bs,
                    transpose=True,
                    transpose_y=True,
                    num_workers=1,
                    pad_idx=PAD_ID,
                    sampler=test_samp,
                    pre_pad=False)

    tensor1 = None
    model.reset()  # todo:bcm - do or dont'?
    with no_grad_context():
        for (x, y) in dl:
            tensor1 = model(x)
    p = tensor1[0]
    
    #     arg = torch.sum(p[:-2],0)
    arg =  p[-1]
#     arg, _  = torch.max(p,dim=0)
#     print(arg.size())
    r = int(torch.argmax(arg))
#     r = int(torch.multinomial(p[-1].exp(), 1))


    while r in [ids_[-1]]: #, 24, BOS_ID,EOS_ID, UNK_ID]:
        arg[r] = -1
        r = int(torch.argmax(arg))

    predicted_ids = [r]
    return predicted_ids


def next_word(ss, model, toks_at_once):
    ids = spm_processor.encode_as_ids(ss)
    return spm_processor.decode_ids(next_tokens(ids, model, toks_at_once))

In [35]:
def next_words_good(ss, lm, n_words, finishers=set([UNK_ID])):
    initial_wip = spm_processor.encode_as_ids(ss)
    wip = initial_wip
    for i in range(n_words):
        wip = wip + next_tokens(wip, lm, 1)

    print(seq(wip).drop(len(initial_wip)))
    return spm_processor.decode_ids(wip)
#         seq(wip).take_while(lambda x: x not in finishers).to_list())

In [36]:
def next_words_bad(ss, lm, n_words, toks_at_once=1):
    wip = ss
    for i in range(n_words):
        wip = wip + " " + next_word(wip, lm, toks_at_once)
    return wip

In [39]:
sentences = [
    "Warszawa to największe", "Te zabawki należą do",
    "Policjant przygląda się", "Na środku skrzyżowania widać",
    "Właściciel samochodu widział złodzieja z",
    "Prezydent z premierem rozmawiali wczoraj o", "Witaj drogi",
    "Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie",
    "Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie",
    "Polscy naukowcy odkryli w Tatrach nowy gatunek istoty żywej. Zwięrzę to przypomina małpę, lecz porusza się na dwóch nogach i potrafi posługiwać się narzędziami. Przy dłuższej obserwacji okazało się, że potrafi również posługiwać się językiem polskim, a konkretnie gwarą podhalańską. Zwierzę to zostało nazwane"
]

In [61]:
for sentence in sentences:
    print("")
    lm.reset()
    print(highlighted(sentence,next_words_bad(sentence, lm, 70)))


Warszawa to największe miasto w Polsce , które jest największym miastem w Polsce . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Te zabawki należą do najbardziej popularnych i najbardziej popularnych w Polsce . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Policjant przygląda się , jak a ntyterrorystyczn y                                                                 

Na środku skrzyżowania widać ślad y a w nim ślad y . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

Właściciel samochodu widział złodzieja z                                                                      

Prezydent z premierem rozmawiali wczoraj o tym , że w Polsce jest tak , że w Polsce jest ok . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

### Without rebuilding embedings from string for every word:

In [62]:
for sentence in sentences:
    print("")
    lm.reset()
    print(highlighted(sentence,next_words_good(sentence, lm, 70)))


[1045, 2, 24, 61, 49, 32625, 188, 4, 7, 6, 0, 10, 4, 17, 7, 22, 0, 7, 5, 0, 3, 7, 2, 0, 24, 7, 10, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0]
Warszawa to największe miasto W Po tymwiecie czasie ,  w ⁇ y , się  nie ⁇   . ⁇   ⁇  W y ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇ 

[558, 2, 24, 61, 174, 32625, 11, 17, 7, 42543, 0, 13, 4, 7, 39, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0]
Te zabawki należą do najbardziej W Po związkuwiecie z się gnij ⁇ e ,  a ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇   , ⁇ 

[4, 2, 24, 61, 49, 32625, 188, 4, 7, 6, 0, 10, 4, 17, 7, 22, 0, 7, 5, 0, 3, 7, 2, 0, 24, 7, 10, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7, 4, 0, 7

## Playground

In [44]:
inpucik = "Mama bardzo dobrze wydała swoje ulubione"
next_words_good(inpucik,lm,30)

[4838, 2, 24, 61, 49, 32625, 188, 4, 7, 6, 0, 10, 4, 17, 7, 22, 0, 7, 5, 0, 3, 7, 2, 0, 24, 7, 10, 0, 7, 4]


'Mama bardzo dobrze wydała swoje ulubione piosenki W Po tymwiecie czasie ,  w ⁇ y , się  nie ⁇   . ⁇   ⁇  W y ⁇   ,'

In [45]:
next_words_bad("Mama bardzo dobrze wydała swoje",lm,30)

'Mama bardzo dobrze wydała swoje ulubione piosenki , które nie są w stanie wytrzymać . . . . . . . . . . . . . . . . . . . . .'

In [52]:
next_words_bad("Policja nie może",lm,70)

'Policja nie może być w stanie w sposób skuteczny i skuteczny przeciwdziała ć wi a                                                          '

In [56]:
next_words_bad("Jest ciepły",lm,70)

'Jest ciepły , ciepły , ciepły , ciepły , ciepły , ciepły , ciepły , ciepły . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .'

In [57]:
next_words_bad("Był bardzo ciepły",lm,70)

'Był bardzo ciepły , ale nie miał ochoty na to , by go                                                            '

In [58]:
next_words_bad("Adam rzucił piłkę burkowi, a ten",lm,70)

'Adam rzucił piłkę burkowi, a ten nie dał rady . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .'

In [60]:
next_words_bad("Pan Premier jest",lm,70)

'Pan premier jest uprzejmy powiedzieć , że w tej chwili jest to niemożliwe . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .'